In [1]:
import pandas as pd
import numpy as np

In [10]:
!pip install sentence-transformers

In [4]:
df = pd.read_csv('databricks2.csv',usecols=['title'])
df.head()

,title
0,jproperties library installation issue
1,Union with Null
2,Spark ReadStream fails to infer schema
3,Unable to execute Notebook commands
4,ARR Customer CVS: Issue - Unable to grant read...


In [5]:
!pip install urlextract

In [6]:
import re
from urlextract import URLExtract
extractor = URLExtract()
import contractions
import nltk
nltk.download('brown')
# dictionary = {'" "':  ' ', ",": ' ','-' : ' ',' " ': ' ','()' : '',
#                   '%' : '','#': '','\[':'','\]': '',
#                   '\\(\\)':'','\*':' ' ,'\|':'',
#                  '!':'','\?':'','\(\)': '','/': '',
#                   '\)' : '', '\(' : '', '"': '', ':':'',"'": '','  +':' ',
#                  }

def preprocess(text):
    
    #text = text.replace(dictionary,regex=True)
    #to lowercase
    text = str(text).lower()
    #remove urls
    urls = extractor.find_urls(text)
    for url in urls:
        text = text.replace(url, '')
    #remove new lines
    text = text.replace('\\n',' ')
    #replacement
    text = text.replace(re.escape("\]\["), "")
    text = text.replace(re.escape("]"), "")
    #remove none meaningful    
    text = re.sub(r'\\x[0-9a-f]{2}', '',text)
    #remove emails
    text = re.sub(r'\S*@\S*\s?',' ',text)
    #remove mentions
    text = re.sub(r'@\S+', ' ', text)
    #contractions
    text = contractions.fix(text)
    #remove hashtags
    text = re.sub(r'@\S+', ' ', text)
    #remove emojis
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    #remove retweets
    text = text.replace(r'rt', '')
    #remove all punct
    text = re.sub('[^A-za-z0-9]', ' ', text)
    #remove extras whitespaces
    text = re.sub('\s+', ' ', text)
    #text = cleaning(text)
    return text

[nltk_data] Downloading package brown to
[nltk_data]     /Users/ankush.singal/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [7]:
df['title'] = df['title'].apply(preprocess)

In [11]:
def remove_shorter_query(data,column,n):
    data[column] = data[column].apply(lambda x:str(x).split(' '))
    data = data[data[column].apply(lambda x:True if len(x) > n else False)]
    data[column] = data[column].apply(lambda x:' '.join(x))
    data = data.reset_index()
    del data["index"]
    
    return data

In [15]:
df = remove_shorter_query(df,"title",2)

/var/folders/7q/zq0k80m94v5408fgfvkddv880000gp/T/ipykernel_65891/127429367.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data[column].apply(lambda x:' '.join(x))


In [16]:
result=df['title'].to_list()

In [18]:
#result

In [27]:
from sentence_transformers import SentenceTransformer,util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [25]:
embeddings = model.encode(result,convert_to_tensor=True)

In [ ]:
# for embedding in embeddings:
#     print(embeddings[:2])

In [29]:
#compute the similarity scores
cosine_scores = util.cos_sim(embeddings, embeddings)
#compute/find the highest similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i] 
                                                             [j]})
#sort the scores in decreasing order 
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(result[i],
                                  result[j], pair['score']))

jobs failing with library not being installed on dataclusters 		 jobs failing with library not being installed on dataclusters 		 Score: 1.0000
css sfmc 2203170040001201 adb cluster hung 		 css sfmc 2203170040001201 adb cluster hung 		 Score: 1.0000
css sfmc 2203170040001201 adb cluster hung 		 css sfmc 2203170040001201 adb cluster hung 		 Score: 1.0000
css sfmc 2203170040001201 adb cluster hung 		 css sfmc 2203170040001201 adb cluster hung 		 Score: 1.0000
unexpected failure while waiting for the cluster 		 unexpected failure while waiting for the cluster 		 Score: 1.0000
2204140050001360 job failure 		 2204140050001360 job failure 		 Score: 1.0000
clusters are holding too many connections 		 clusters are holding too many connections 		 Score: 1.0000
attributeerror module lib has no attribute x509_v_flag_cb_issuer_check  		 attributeerror module lib has no attribute x509_v_flag_cb_issuer_check  		 Score: 1.0000
attributeerror module lib has no attribute x509_v_flag_cb_issuer_check  		

In [ ]:
#https://towardsdatascience.com/an-intuitive-explanation-of-sentence-bert-1984d144a868

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [embeddings[0]],
     embeddings[1:]
)
#https://github.com/jamescalam/transformers/blob/main/course/similarity/04_sentence_transformers.ipynb
#https://github.com/jamescalam/transformers/blob/main/course/similarity/03_calculating_similarity.ipynb